In [24]:
import sys
sys.path[:0] = ['../../ref/sw/compiler', '../basics',"../../hw/base"]

from parser import IRParser
from backend import CompilerBackend
from asm_interp import AsmInterp


In [25]:
p = IRParser()
cb = CompilerBackend()
ai = AsmInterp()

In [26]:
cb.funcs(p('''
    /* main func */
    main(0) = if pos(pow2($0)) then 1 else 0

    /* power ^2 function */
    pow2(1) = $0 * $0

    /* positive number */
    pos(1) = if 0 < $0 then 1 else 0
'''))

In [27]:
cb.code

['main',
 ('PUSH', 'main:0'),
 ('PUSH', 'main:1'),
 ('PUSH', 2),
 ('POP', 2),
 ('ALU', 'SUB'),
 ('JMP', 'pow2'),
 'main:1',
 ('JMP', 'pos'),
 'main:0',
 ('POP', 1),
 ('JZ', 'main:2'),
 ('PUSH', 1),
 ('JMP', 'main:3'),
 'main:2',
 ('PUSH', 0),
 'main:3',
 ('POP', 2),
 ('RET', 1),
 'pow2',
 ('DUP', 0),
 ('DUP', 1),
 ('POP', 2),
 ('ALU', 'MUL'),
 ('YANK', (1, 1)),
 ('POP', 2),
 ('RET', 1),
 'pos',
 ('PUSH', 0),
 ('DUP', 1),
 ('POP', 2),
 ('ALU', 'LT'),
 ('POP', 1),
 ('JZ', 'pos:0'),
 ('PUSH', 1),
 ('JMP', 'pos:1'),
 'pos:0',
 ('PUSH', 0),
 'pos:1',
 ('YANK', (1, 1)),
 ('POP', 2),
 ('RET', 1),
 'main',
 ('PUSH', 'main:4'),
 ('PUSH', 'main:5'),
 ('PUSH', 2),
 ('POP', 2),
 ('ALU', 'SUB'),
 ('JMP', 'pow2'),
 'main:5',
 ('JMP', 'pos'),
 'main:4',
 ('POP', 1),
 ('JZ', 'main:6'),
 ('PUSH', 1),
 ('JMP', 'main:7'),
 'main:6',
 ('PUSH', 0),
 'main:7',
 ('POP', 2),
 ('RET', 1),
 'pow2',
 ('DUP', 0),
 ('DUP', 1),
 ('POP', 2),
 ('ALU', 'MUL'),
 ('YANK', (1, 1)),
 ('POP', 2),
 ('RET', 1),
 'pos',
 ('PU

In [26]:
p = IRParser()
cb = CompilerBackend()
ai = AsmInterp()
cb.funcs(p('''/*
/*
main =
    let l = 11, h = 20
    horiz l (h + 1) l;
    vert h (l + 1) (h + 1);
    horiz (h - 1) l h;
    vert l h l
*/
main(0) =
    /*ignore square 0 10 20;*/
    ignore spiral;
    0

/* square(y, x, side): draws a square of given side from (x, y) */
square(3) =
    /* top edge: horiz x → x+side at y */
    ignore horiz $1 ($1 + $0) $2;

    /* bottom edge: horiz x → x+side at y+side */
    ignore horiz $1 ($1 + $0 + 1) ($2 + $0);

    /* left edge: vert y → y+side at x */
    ignore vert $1 $2 ($2 + $0);

    /* right edge: vert y → y+side at x+side */
    ignore vert ($1 + $0) $2 ($2 + $0);

    0
    
/* draws a square spiral on the screen */
spiral(0) = 
    square 15 15 1;
    square 14 14 3;

    horiz 3 29 3;
    vert  29 3 29;
    horiz 29 3 28;
    vert  3 28 7;

    horiz 3 25 7;
    vert  25 7 24;
    horiz 25 7 24;
    vert  7 24 15;

    horiz 7 15 15;
    0
memset_skip(4) =
    if 0 < $1 then mem_poke $0 $3; .memset_skip ($3 + $2) $2 ($1 - 1) $0 else 0
    
/*
# memor_skip start skip cnt value : unit =
#   if 0 < cnt then mem_poke (value | mem_peek(start)) start;
#                   memor_skip (start + skip) skip (cnt - 1) value
#              else ()
*/
memor_skip(4) =
    if 0 < $1 then mem_poke ($0 | mem_peek $3) $3; .memor_skip ($3 + $2) $2 ($1 - 1) $0 else 0

/*
# block x y =
#   let mask = (x & 1 ? 0xff00 : 0x00ff) in 
#   memor_skip (0xa000 + 0x80 * y + (x >> 1)) 0x10 0x08 mask
*/
block(2) =
    (if $1 & 1 then 0xff00 else 0x00ff);
    memor_skip (0xa000 + 0x80 * $0 + ($1 >> 1)) 0x10 0x08 $-1

/*
# memor_shr_skip start skip cnt value shft : unit =
#   if 0 < cnt then mem_poke ((value << shft) | mem_peek(start)) start;
#                   memor_shr_skip (start + skip) skip (cnt - 1) (value >> 1)
#              else ()
*/
memor_shr_skip(5) =
    if 0 < $2 then ignore mem_poke (($1 << $0) | mem_peek $4) $4;
                   memor_shr_skip ($4 + $3) $3 ($2 - 1) ($1 >> 1) $0
              else 0

/*
# memor_shl_skip start skip cnt value shft : unit =
#   if 0 < cnt then mem_poke ((value >> shft) | mem_peek(start)) start;
#                   memor_shr_skip (start + skip) skip (cnt - 1) (value << 1)
#              else ()
*/
memor_shl_skip(5) =
    if 0 < $2 then ignore mem_poke (($1 >> $0) | mem_peek $4) $4;
                   memor_shl_skip ($4 + $3) $3 ($2 - 1) ($1 << 1) $0
              else 0

/* horiz x0 x1 y  */
horiz(3) =
    (if $2 < $1 then 1 else (if $1 < $2 then 0xffff else 0));
    if $-1 then ignore block $2 $0; .horiz ($2 + $-1) $1 $0
           else 0

/* vert x y0 y1  */
vert(3) =
    (if $1 < $0 then 1 else (if $0 < $1 then 0xffff else 0));
    if $-1 then ignore block $2 $1; .vert $2 ($1 + $-1) $0
           else 0


    
'''))


In [ ]:
stack = Array('stack', IntSort(), IntSort())
mem = Array('mem', IntSort(), IntSort())
sp = Int('sp')
r0 = Int('r0')
r1 = Int('r1')
state_vars = [stack, sp, r0, r1, mem]

a, b = input_vars = Ints('a b')
def create_chc(program, input_vars, preCondition, postCondition):
    sigma = [*input_vars, *state_vars]
    U = {i: Function(f"U{i}", *(v.sort() for v in sigma), BoolSort())
         for i in range(len(program) + 1)}
    labels = _index_labels(program)
    chc = []

    # Initial precondition
    chc.append(Implies(preCondition, U[0](sigma)))

    for i in range(len(program)):
        instruction = program[i]

        match instruction[0]:
            case "PUSH":
                if instruction[1] in labels:
                    chc.append(Implies(
                        U[i](sigma),
                        U[i+1](
                            *input_vars,
                            Store(stack, sp, labels[instruction[1]]),
                            sp + 1,
                            r0, r1, mem
                        )
                    ))
                else:
                    chc.append(Implies(
                        U[i](sigma),
                        U[i+1](
                            *input_vars,
                            Store(stack, sp, instruction[1]),
                            sp + 1,
                            r0, r1, mem
                        )
                    ))

            case "POP":
                if instruction[1] == 1:
                    v0 = Select(stack, sp - 1)
                    chc.append(Implies(
                        U[i](sigma),
                        U[i+1](*input_vars, stack, sp - 1, v0, r1, mem)
                    ))
                elif instruction[1] == 2:
                    v0 = Select(stack, sp - 2)
                    v1 = Select(stack, sp - 1)
                    chc.append(Implies(
                        U[i](sigma),
                        U[i+1](*input_vars, stack, sp - 2, v0, v1, mem)
                    ))

            case "ALU":
                match instruction[1]:
                    case "ADD":
                        v = Int(f"alu_addv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == r0 + r1),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))
                    case "SUB":
                        v = Int(f"alu_subv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == r0 - r1),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))
                    case "MUL":
                        v = Int(f"alu_mulv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == r0 * r1),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))
                    case "NEG":
                        v = Int(f"alu_negv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == -r0),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))
                    case "AND":
                        v = Int(f"alu_andv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == If(And(r0 != 0, r1 != 0), 1, 0)),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))
                    case "OR":
                        v = Int(f"alu_orv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == If(Or(r0 != 0, r1 != 0), 1, 0)),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))
                    case "NOT":
                        v = Int(f"alu_notv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == If(r0 == 0, 1, 0)),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))
                    case "SHL":
                        v = Int(f"alu_shlv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == r0 << r1),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))
                    case "SHR":
                        v = Int(f"alu_shrv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == r0 >> r1),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))
                    case "LT":
                        v = Int(f"alu_ltv_{i}")
                        chc.append(Implies(
                            And(U[i](sigma), v == If(r0 < r1, 1, 0)),
                            U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                        ))

            case "STOR":
                chc.append(Implies(
                    U[i](sigma),
                    U[i+1](*input_vars, stack, sp, r0, r1, Store(mem, r1, r0))
                ))

            case "LOAD":
                v = Int(f"mem_load_{i}")
                chc.append(Implies(
                    And(U[i](sigma), v == Select(mem, r0)),
                    U[i+1](*input_vars, Store(stack, sp, v), sp + 1, r0, r1, mem)
                ))

            case "JMP":
                target = instruction[1]
                if isinstance(target, int):
                    jmp_target = target
                elif isinstance(target, str):
                    label = target[:-1] if target.endswith(":") else target
                    jmp_target = labels.get(label, label)
                else:
                    raise TypeError(f"Unexpected JMP target type: {target}")

                if jmp_target == "mem_poke":
                    addr = Select(stack, sp - 2)
                    val = Select(stack, sp - 1)
                    new_mem = Store(mem, addr, val)
                    chc.append(Implies(
                        U[i](sigma),
                        U[i+1](*input_vars, stack, sp, r0, r1, new_mem)
                    ))

                elif jmp_target == "mem_peek":
                    addr = Select(stack, sp - 1)
                    val = Int(f"mem_peek_val_{i}")
                    chc.append(Implies(
                        And(U[i](sigma), val == Select(mem, addr)),
                        U[i+1](
                            *input_vars,
                            Store(stack, sp, val),
                            sp + 1,
                            r0, r1,
                            mem
                        )
                    ))

                else:
                    if not isinstance(jmp_target, int):
                        raise ValueError(f"Unknown jump target: {jmp_target}")
                    chc.append(Implies(
                        U[i](sigma),
                        U[jmp_target](*input_vars, stack, sp, r0, r1, mem)
                    ))

            case "JZ":
                target = instruction[1]
                jmp_target = target if isinstance(target, int) else labels[target[:-1]]
                chc.append(Implies(
                    And(U[i](sigma), r0 == 0),
                    U[jmp_target](*input_vars, stack, sp, r0, r1, mem)
                ))

            case "JNZ":
                target = instruction[1]
                jmp_target = target if isinstance(target, int) else labels[target[:-1]]
                chc.append(Implies(
                    And(U[i](sigma), r0 != 0),
                    U[jmp_target](*input_vars, stack, sp, r0, r1, mem)
                ))

            case "RET":
                if instruction[1] == 0:
                    chc.append(Implies(
                        U[i](sigma),
                        U[r0](*input_vars, stack, sp, r0, r1, mem)
                    ))
                elif instruction[1] == 1:
                    chc.append(Implies(
                        U[i](sigma),
                        U[r0](*input_vars, Store(stack, sp, r1), sp + 1, r0, r1, mem)
                    ))

            case "DUP":
                offset = instruction[1]
                v = Int(f"dupv_{i}")
                chc.append(Implies(
                    And(U[i](sigma), v == Select(stack, sp - 1 - offset)),
                    U[i+1](
                        *input_vars,
                        Store(stack, sp, v),
                        sp + 1,
                        r0, r1,
                        mem
                    )
                ))

            case "YANK":
                offset, count = instruction[1]
                new_stack = stack
                rmin = sp - offset - count
                for k in range(offset):
                    tgt = rmin + k
                    src = tgt + count
                    new_stack = Store(new_stack, tgt, Select(stack, src))
                chc.append(Implies(
                    U[i](sigma),
                    U[i+1](
                        *input_vars,
                        new_stack,
                        sp - count,
                        r0, r1,
                        mem
                    )
                ))

    # Postcondition
    chc.append(Implies(U[len(program)](sigma), postCondition))
    return CHCs(chc)

In [37]:
chc = create_chc(cb.code, [], True, True)
chc

KeyError: r0